In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Кредитный Скоринг
#### Автор: Галкин Александр
#### Цель проета: построение модели, предсказывающей вероятность дефолта по кредиту на основе данных по клиенту


## Импорт библиотек

In [ ]:
import os
import sklearn
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm
import catboost as cb
import xgboost as xgb
import optuna
from datetime import datetime
import warnings

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score,plot_roc_curve
from sklearn.metrics import classification_report, precision_score, recall_score, plot_precision_recall_curve
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import statsmodels.api as sm
from imblearn.over_sampling import RandomOverSampler


from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.inspection import permutation_importance

warnings.filterwarnings("ignore")

%matplotlib inline

## Загрузка данных

#### Описание полей:

client_id - идентификатор клиента

education - уровень образования

sex - пол заемщика

age - возраст заемщика

car - флаг наличия автомобиля

car_type - флаг автомобиля иномарки

decline_app_cnt - количество отказанных прошлых заявок

good_work - флаг наличия “хорошей” работы

bki_request_cnt - количество запросов в БКИ

home_address - категоризатор домашнего адреса

work_address - категоризатор рабочего адреса

income - доход заемщика

foreign_passport - наличие загранпаспорта

sna - связь заемщика с клиентами банка

first_time - давность наличия информации о заемщике

score_bki - скоринговый балл по данным из БКИ

region_rating - рейтинг региона

app_date - дата подачи заявки

default - флаг дефолта по кредиту

In [ ]:
path = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(path +'/train.csv')
test = pd.read_csv(path +'test.csv')
sample = pd.read_csv(path +'/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
for col in train.columns:
    print(col, train[col].nunique())

Получаем:
* 5 бинарных признаков: sex', 'car', 'car_type', 'good_work', 'foreign_passport'
* 6 категориальных: 'education', 'work_address', 'home_address', 'sna', 'first_time', 'region_rating'
* 5 числовых: 'age', 'decline_app_cnt', 'score_bki', 'income', 'bki_request_cnt'
* признак 'app_date' нужно обработать отдельно
* признак 'region_rating' можно сделать как числовым, так и категориальным(всего 7 значений). Пусть будет категориальным.

Создадим списки призников по категориям:

In [ ]:
#Бинарные переменные
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

#Категориальные переменные
cat_cols = ['education', 'work_address', 'home_address', 'sna', 'first_time', 'region_rating'] 

#Числовые переменные
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'income', 'bki_request_cnt'] 

In [ ]:
sample.head()

### Первые выводы по данным:
1. Нулевые записи есть только в поле 'education'
2. В таблицу ответов записывается соответсвие id и вероятности дефолта
3. Нужно объединить test и train таблицы для одновременной предобработки даннных.  Для последующег разединения таблиц добавим столбец-флаг  

In [ ]:
# объединим таблицы
train['train'] = 1   # флаг обучающей выборки
test['train'] = 0    # флаг тестовой выборки
test['default'] = -1 # чтобы не было NaNов 
data = train.append(test, sort=False).reset_index(drop=True)

# Exploratory data analysis + Feature engineering

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
train['default'].hist()

Классы сильно разбалансированы. Может негативно повлиять наобучение модели.

## Числовые признаки
Посмотрим на гистограммы числовых признаков

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15,10))

for i,col in enumerate(num_cols):
    sns.distplot(data[col], kde=False, ax=axes.flat[i])
    plt.title(i)

У трёх признаков ('decline_app_cnt', 'bki_request_cnt', 'income') есть большие "хвосты". Вероятно, это выбросы. Попробуем логарифмировать данныев этих столбцах.

In [ ]:
to_log = ['decline_app_cnt', 'bki_request_cnt', 'income']
for col in to_log:
    data[col] = np.log(data[col] + 1)

In [ ]:
# Посмотрим теперь на гистограммы этих столбцов:
fig, axes = plt.subplots(3, 1, figsize=(15,10))

for i,col in enumerate(to_log):
    sns.distplot(data[col], kde=False, ax=axes.flat[i])
    plt.title(i)

Распределение дохода стало похоже на нормальное. Хвосты на остальных стали заметно скромнее. Оставим так.

## Категориальные признаки

In [ ]:
# Теперь заполним пропуски в столбце "education" самым распространенным значением
data.education.value_counts()

In [ ]:
data['education'].fillna(value='SCH', inplace = True)

Заменим строковые данные столбца  "education" на числовые

In [ ]:
label_encoder = LabelEncoder()
data['education'] = label_encoder.fit_transform(data['education'])
data.head()

## Признак DateTime

In [ ]:
data['app_date'].describe()

In [ ]:
data['app_date'].min()

In [ ]:
data['app_date'].max()

120 уникальных сначений с 1 апреля по 31 марта 2014 года.
Заменим значения в столбце 'app_date' на целые числа - количество дней с первого обращения

In [ ]:
data['app_date'] = pd.to_datetime(data.app_date)
data['app_date'] = data['app_date'].apply(lambda x: (x - data['app_date'].min()).days)

In [ ]:
data.app_date

In [ ]:
num_cols.append('app_date')

## Бинарные признаки

In [ ]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for col in bin_cols:
    data[col] = label_encoder.fit_transform(data[col])

In [ ]:
# Посмотрим на нашу таблицу
data.head()

In [ ]:
data.info()

## Построим матрицу корреляции
Теперь у нас все признаки числовые. Можно посмотреть как они коррелируют между собой.

In [ ]:
plt.rcParams['figure.figsize'] = (20,15)
sns.heatmap(data[data['train'] == 1].corr().abs(), vmin=0, vmax=1, annot=True)

Больше всего наша целевая переменнаа default коррелирует с призниками score_bki(!), decline_app_cnt, sna и first_time. 

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
cols = data.columns
dd = data[data['train']==1]
imp_num = pd.Series(f_classif(dd[num_cols], dd['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость числовых признаков (F test)')

In [ ]:
cat_bin = cat_cols + bin_cols
#print(cat_bin)
dd = data[data['train']==1]
imp_cat = pd.Series(mutual_info_classif(dd[cat_bin], dd['default'], discrete_features =True), cat_bin)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh', title = 'Значимость бин. и категор. переменных по Mutual information test')

## Dummy variables
Создадим из категориальных признаков dummy переменные

In [ ]:
df = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

In [ ]:
df

## Стандартизируем числовые призники

In [ ]:
df

In [ ]:
# стандартизируем числовые признаки 
df[num_cols] = pd.DataFrame(StandardScaler().fit_transform(df[num_cols]), columns = df[num_cols].columns)

In [ ]:
df

In [ ]:
df

## Обучение модели

In [ ]:
# Разделим наш датасет на тейн и тест по флагу 'train'. Также уберем столбцы 'id' и целевую переменную.
train_df = df.query('train == 1').drop(['train', 'client_id'], axis=1)
test_df = df.query('train == 0').drop(['train'], axis=1)

y = train_df.default.values            
X = train_df.drop(['default'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=76)

###  Для начала обучим логистическую регрессию

In [ ]:
lr = LogisticRegression(max_iter = 5000)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
probs = lr.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
print('accuracy_score:',accuracy_score(y_test,y_pred))
print('precision_score:',precision_score(y_test,y_pred))
print('recall_score:',recall_score(y_test,y_pred))
print('f1_score:',f1_score(y_test,y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [ ]:
sns.displot(y);

In [ ]:
# Данные сильно разбалансированы. Модель настойчиво предсказывает "НЕ ДЕФОЛТНЫЙ" класс.

In [ ]:
# prepare data for oversampling
oversample = RandomOverSampler(sampling_strategy=1)
X_os, y_os = oversample.fit_resample(X, y)

In [ ]:
sns.displot(y_os);

In [ ]:
X_train_os, X_test_os, y_train_os, y_test_os = train_test_split(X_os, y_os, test_size=0.2, random_state=76)

In [ ]:
lr_os = LogisticRegression(max_iter = 5000)

lr_os.fit(X_train_os, y_train_os)
y_pred_os = lr_os.predict(X_test_os)

In [ ]:
probs = lr.predict_proba(X_test_os)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test_os, probs)
roc_auc = roc_auc_score(y_test_os, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
print('accuracy_score:',accuracy_score(y_test_os,y_pred_os))
print('precision_score:',precision_score(y_test_os,y_pred_os))
print('recall_score:',recall_score(y_test_os,y_pred_os))
print('f1_score:',f1_score(y_test_os,y_pred_os))

In [ ]:
cm = confusion_matrix(y_test_os, y_pred_os)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

Все параметры стали ровно по 2./3(две трети):
* accuracy: 0.6732373598230569
* precision: 0.6677229865512444
* recall: 0.6769845623383748
* f1_score: 0.6723218802288027

Попробуем обучить другую модель.

In [ ]:
xgb = XGBClassifier(learning_rate=0.001, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
xgb_pred = xgb.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, xgb_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [ ]:
balance = len(df[df['default'] == 1]) / len(df[df['default'] == 0])

In [ ]:
balance

In [ ]:

xgb = XGBClassifier(learning_rate=0.001, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
print('accuracy_score:',accuracy_score(y_test,y_pred))
print('precision_score:',precision_score(y_test,y_pred))
print('recall_score:',recall_score(y_test,y_pred))
print('f1_score:',f1_score(y_test,y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

Попробуем обучить модель на oversampling данных.

In [ ]:
xgb = XGBClassifier(learning_rate=0.001, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
xgb.fit(X_train_os, y_train_os)

In [ ]:
y_pred_os = xgb.predict(X_test_os)

In [ ]:
print('accuracy_score:',accuracy_score(y_test_os,y_pred_os))
print('precision_score:',precision_score(y_test_os,y_pred_os))
print('recall_score:',recall_score(y_test_os,y_pred_os))
print('f1_score:',f1_score(y_test_os,y_pred_os))

In [ ]:
cm = confusion_matrix(y_test_os, y_pred_os)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [ ]:
id_test = test_df['client_id']
id_test

In [ ]:
test_dff = test_df.drop(['client_id', 'default'], axis=1)

In [ ]:
result = xgb.predict_proba(test_dff)

In [ ]:


submission = pd.DataFrame({'client_id': id_test,
                           'default': xgb.predict_proba(test_dff)[:, 1]})
submission.to_csv('submission.csv', index=False)

submission